In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
data = pd.read_csv('../data/raw/data.csv')

In [3]:
data.columns

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name', 'game_date', 'matchup', 'opponent', 'shot_id'],
      dtype='object')

### Get Consecutive Shot Made

In [4]:
# sort by game_id
# sort by game_event_id
# add new column, consec_shot_made, init to 0
# shift gameid by 1 to get prev game id and prev_shot_flag
# if prev_game_id == game_id, check shot_flag
# 

#data['game_ev_id'] = data['game_id'].astype(str) + '_' + data['game_event_id'].astype(str)
gamedata = data.sort_values(by=['game_id','game_event_id'])
gamedata['consec_shot_made'] = 0
gamedata['prev_game_id'] = gamedata['game_id'].shift(1)
gamedata['prev_game_id'].fillna(gamedata['game_id'], inplace=True)
gamedata['prev_game_id'] = gamedata['prev_game_id'].astype('int64')
gamedata['shot_made_flag'].fillna(-1, inplace=True)

In [5]:
gamedata[['game_event_id','prev_game_id','game_id','shot_made_flag','consec_shot_made']]
print(gamedata['shot_zone_area'].value_counts())
print(gamedata['shot_zone_range'].value_counts())

Center(C)                13455
Right Side Center(RC)     4776
Right Side(R)             4588
Left Side Center(LC)      4044
Left Side(L)              3751
Back Court(BC)              83
Name: shot_zone_area, dtype: int64
Less Than 8 ft.    9398
16-24 ft.          8315
8-16 ft.           6626
24+ ft.            6275
Back Court Shot      83
Name: shot_zone_range, dtype: int64


In [6]:
# calculate participation in the game - see if kobe is in the flow
# for lack of data, only measure if event is 'shooting' event - distance between game_event_id
# gamedata['event_between_attempt']
#gamedata['game_id'].value_counts()
#gamedata[['prev_game_id','game_id', 'game_event_id']]
gamedata['prev_game_event_id'] = gamedata['game_event_id'].shift(1)
gamedata['prev_game_event_id'].fillna(gamedata['game_event_id'], inplace=True)
gamedata['prev_game_event_id'] = gamedata['prev_game_event_id'].astype('int64')

In [12]:
gamedata[['prev_game_id','game_id', 'prev_game_event_id','game_event_id']]

,prev_game_id,game_id,prev_game_event_id,game_event_id
0,20000012,20000012,10,10
1,20000012,20000012,10,12
2,20000012,20000012,12,35
3,20000012,20000012,35,43
4,20000012,20000012,43,155
5,20000012,20000012,155,244
6,20000012,20000012,244,251
7,20000012,20000012,251,254
8,20000012,20000012,254,265
9,20000012,20000012,265,294


In [22]:
# def get_flow(prev_game_id, game_id, prev_game_event_id, game_event_id):
def get_flow(row):
    ret = 0
    prev_game_id = row['prev_game_id']
    game_id = row['game_id']
    prev_game_event_id = row['prev_game_event_id']
    game_event_id = row['game_event_id']
    if game_id == prev_game_id:
        ret = game_event_id - prev_game_event_id
    return ret

In [23]:
gamedata['flow'] = gamedata.apply(get_flow, axis=1)

In [27]:
gamedata[['prev_game_id','game_id','prev_game_event_id','game_event_id','flow']].head(3)

,prev_game_id,game_id,prev_game_event_id,game_event_id,flow
0,20000012,20000012,10,10,0
1,20000012,20000012,10,12,2
2,20000012,20000012,12,35,23
